tar -C /tmp/ayu -zvxf xyz.tar.gz

In [1]:
import os,sys,pickle,time
import pandas as pd
import numpy as np
import subprocess
from multiprocessing import Pool 
import shutil
from tqdm import tqdm

In [27]:
tgz_dir = "../../RAID5/pcap/"
decompress_dir = "../../NAS2_RAID6/pcap_process/"
all_dates = next(os.walk(tgz_dir))[1]
all_dates = sorted(all_dates)
all_dates

['2019_12_29',
 '2019_12_30',
 '2019_12_31',
 '2020_01_01',
 '2020_01_02',
 '2020_01_03',
 '2020_01_04',
 '2020_01_05',
 '2020_01_06',
 '2020_01_07',
 '2020_01_08',
 '2020_01_09',
 '2020_01_10',
 '2020_01_11',
 '2020_01_12',
 '2020_01_13',
 '2020_01_14',
 '2020_01_15',
 '2020_01_16',
 '2020_01_17',
 '2020_01_18',
 '2020_01_19',
 '2020_01_20',
 '2020_01_21',
 '2020_01_22',
 '2020_01_23',
 '2020_01_24',
 '2020_01_25',
 '2020_01_26',
 '2020_01_27',
 '2020_01_28',
 '2020_01_29',
 '2020_01_30',
 '2020_01_31',
 '2020_02_01',
 '2020_02_02',
 '2020_02_03',
 '2020_02_04',
 '2020_02_05',
 '2020_02_06',
 '2020_02_07',
 '2020_02_08',
 '2020_02_09',
 '2020_02_10',
 '2020_02_11',
 '2020_02_12',
 '2020_02_13',
 '2020_02_14',
 '2020_02_15',
 '2020_02_16',
 '2020_02_17',
 '2020_02_18',
 '2020_02_19',
 '2020_02_20',
 '2020_02_21',
 '2020_02_22',
 '2020_02_23',
 '2020_02_24',
 '2020_02_25',
 '2020_02_26',
 '2020_02_27',
 '2020_02_28',
 '2020_02_29',
 '2020_03_01',
 '2020_03_02',
 '2020_03_03',
 '2020_03_

In [59]:
taken = []
def parallel_untar(in_file,out_dir):
    test_dir = out_dir.split('/')
    test_dir = test_dir[:3]
    test_dir = "/".join(test_dir)
    total, used, free = shutil.disk_usage(test_dir)
    if (free<total*0.01) or (free // (2**30) < 130):
#         cmd = ['rmdir',out_dir]
#         p = subprocess.Popen(cmd, stderr=subprocess.PIPE, stdout=subprocess.PIPE, shell=False)
#         out, err = p.communicate()
#         print("!Error:", err, '!')
        print('Disk not enough space ERR:',in_file,out_dir)
        return out_dir
    else:
        dt = out_dir.split("/")[-4]
        int_dt = int("".join(dt.split("_")))
        
        
        #cat pcap_2020_04_18_part* > pcap_2020_04_18.tar.gz # 合併
        #cat pcap_2020_04_18_part* | tar -C out_dir -zxvf #解壓，02_24號開號開始分part，以前都是只有一個
        in_prefix = "/".join(in_file.split('/')[:-1]) + '/'
        if (int_dt >20200223)and(in_prefix not in taken):
            taken.append(in_prefix)
            prefix = "/".join(out_dir.split('/')[:-2]) + '/'
            out_dir = prefix + "pcap_"+dt+'_dir/'
            if not os.path.exists(out_dir):
                os.makedirs(out_dir, exist_ok=True)              
            cmd = ["cat "+in_prefix+"pcap_"+dt+"_part* | "+"tar -C "+out_dir+" -zxv"]
            p = subprocess.Popen(cmd, stderr=subprocess.PIPE, stdout=subprocess.PIPE, shell=True)
        else:
            if not os.path.exists(out_dir):
                os.makedirs(out_dir, exist_ok=True)            
            cmd = ["tar","-C", out_dir,
               "-zvxf", in_file]
            p = subprocess.Popen(cmd, stderr=subprocess.PIPE, stdout=subprocess.PIPE, shell=False)
        out, err = p.communicate()
        print("!Error:", err, '! Done:',out_dir)
    

In [39]:
in_file_paths[0].split('/')[:-1]

['..', '..', 'RAID5', 'pcap', '2020_03_07', '中嘉寬頻']

In [25]:
kk = out_dir_paths[0]#.split("/")[-4]
# "/".join(kk.split('/')[:-2]) + '/'
kk

'../../NAS2_RAID6/pcap_process/2020_03_07/中嘉寬頻/pcap_2020_03_07_part1_dir/'

In [10]:
int("".join(out_dir_paths[0].split("/")[-4].split("_"))) >20200223

True

## For ISP's Pcap
parallel by function name

In [60]:
# %%time
in_file_paths = []
out_dir_paths = []
days = ['2020_03_07'] #指定日期，清空掉就是全部日子
isps = ['中華電信','中嘉寬頻','亞太電信','凱擘','台固媒體','台灣之星','台灣固網','台灣基礎開發','台灣大哥大',
       '台灣碩網','遠傳電信']
# isps = ['台灣固網','台固媒體']

now_taken = []
for date in tqdm(all_dates):
    if date not in days:
        continue
    int_dt = int("".join(date.split("_")))
    pcap_path = tgz_dir + date + '/'
    isp_names = next(os.walk(pcap_path))[1]
    take_isps = []
    for isp in isp_names:
        if isp in isps:
            take_isps.append(isp)
    for isp in take_isps:
        file_names = next(os.walk(pcap_path+isp))[2]
        file_names = list(filter(lambda f: f.endswith(".tar.gz"), file_names))
        if len(file_names)>1 and int("".join(date.split("_"))) <=20200223:
            print('WARNING: more than 1 pcap tgz file in',pcap_path+isp+'/') #可能分part
        for file_name in file_names:
            pcap_path_ = pcap_path+isp+'/'+file_name
            prefix = "/".join(pcap_path_.split('/')[:-1]) + '/'
            if (int_dt >20200223)and(prefix in now_taken):    
                continue
            now_taken.append(prefix)
            output_path = decompress_dir+date+'/'+isp+'/'+file_name.replace('.tar.gz','_dir')+'/'
            in_file_paths.append(pcap_path_)
            out_dir_paths.append(output_path)

assert len(in_file_paths) == len(out_dir_paths)
assert in_file_paths[0].split('/')[-3] == out_dir_paths[0].split('/')[-4]
assert in_file_paths[-1].split('/')[-2] == out_dir_paths[-1].split('/')[-3]
len(in_file_paths)

100%|██████████| 70/70 [00:00<00:00, 1352.74it/s]


11

In [48]:
# date
int_dt = int("".join(date.split("_")))
int_dt

20200307

In [56]:
in_file = in_file_paths[0]
dt = out_dir_paths[0].split("/")[-4]
in_prefix = "/".join(in_file.split('/')[:-1]) + '/'
in_prefix+"pcap_"+dt+"_part* | "+"tar -C "

'../../RAID5/pcap/2020_03_07/中嘉寬頻/pcap_2020_03_07_part* | tar -C '

In [18]:
# #create new dir for decompress
# pcap_path = root_dir + name + '/pcap/'
# file_names = next(os.walk(pcap_path))[2]
# file_names = list(filter(lambda f: f.endswith(".tar.gz"), file_names))
# new_dir_names = [x.replace('.tar.gz','_dir') for x in file_names]
# for dir_name in new_dir_names:
#     path = pcap_path+dir_name
#     try:
#         if days != []:
#             for day in days:
#                 if day in path:
#                     os.makedirs(path, exist_ok=True)
#                     paths.append(path)
#         else:
#             os.makedirs(path, exist_ok=True)
#             paths.append(path)
#     except NameError:
#         os.makedirs(path, exist_ok=True)
#         paths.append(path)

# paths

In [ ]:
%%time
load_start = time.time()
pool_result = []
pool = Pool(processes=3) #processes=21
for path_in,dir_out in zip(in_file_paths,out_dir_paths):
    r = pool.apply_async(parallel_untar,args=(path_in,dir_out))
    pool_result.append(r)
pool.close()
pool.join()
for r in pool_result:
    print('return:',r.get())
load_end = time.time() - load_start
print("Decompress Time:", '{:02f}:{:02f}:{:02f}'.format(load_end // 3600, (load_end % 3600 // 60), load_end % 60))

!Error: b'tar: ./snort.log.1583498489: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1583497458: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1583503193: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1583509490: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1583499081: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1583490922: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1583497910: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1583492324: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1583510119: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1583501453: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1583492778: Cannot chan

!Error: b'tar: ./snort.log.1583508342: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1583497567: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1583506449: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1583493973: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1583491293: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1583490537: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1583504364: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1583503159: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1583490290: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1583488809: Cannot change ownership to uid 0, gid 0: Operation not permitted\ntar: ./snort.log.1583502382: Cannot chan

In [28]:
r.get()

FileNotFoundError: [Errno 2] No such file or directory: '../data_hdd4t/pcap_process/2020_02_01/中華電信/pcap_2020_02_01_dir/'

In [29]:
not os.path.exists(dir_out)

True

### remove folder recursively

In [34]:
for name in tqdm(company_names):
#     if name == '中華電信': #指定電信公司
#         continue
    pcap_path = root_dir + name + '/pcap/'
    dir_names = next(os.walk(pcap_path))[1]
    if dir_names != []:
        for folder in dir_names:
            shutil.rmtree(pcap_path+folder+'/')
            print(pcap_path+folder+'/')

  0%|          | 0/11 [00:00<?, ?it/s]

../data_hdd4t/中華電信/pcap/snort.2020-01-04_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-10_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-09_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-02_dir/


100%|██████████| 11/11 [00:04<00:00,  2.25it/s]

../data_hdd4t/中華電信/pcap/snort.2020-01-11_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-08_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-05_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-06_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-07_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-03_dir/
../data_hdd4t/中華電信/pcap/snort.2020-01-01_dir/


## For ISPs' honeypot

In [22]:
paths = []
days = ['2020-01-09','2020-01-10','2020-01-11'] #指定日期，清空掉就是全部日子
for name in tqdm(company_names):
    if name != '中華電信': #指定電信公司，註解掉就是全部
        continue
#     print('=== Running:',name,'===')
    hp_path = root_dir + name + '/honeypot/'
    date_dir_names = next(os.walk(hp_path))[1]
    if days != []:
        temp = []
        for date in date_dir_names:
            for day in days:
                if day == date:
                    temp.append(date)
        date_dir_names = temp
    else:
        pass
    for date in date_dir_names:
        hp_date_path = hp_path + date +'/'
        hp_names_dir = next(os.walk(hp_date_path))[1]
        for name in hp_names_dir:
            file_names = next(os.walk(hp_date_path+name+'/'))[2]
            file_names = list(filter(lambda f: f.endswith(".tar.gz"), file_names))
            new_dir_names = [x.replace('.tar.gz','_dir') for x in file_names]
            for dir_name in new_dir_names:
                path = hp_date_path+name+'/'+dir_name
                os.makedirs(path, exist_ok=True)
                paths.append(path)
        
paths

100%|██████████| 11/11 [00:00<00:00, 420.87it/s]


['../data_hdd4t/中華電信/honeypot/2020-01-10/cowrie/cowrie_log_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/cowrie/cowrie_file_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/amun/amun_log_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/amun/amun_file_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/glastopf/glastopf_file_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/glastopf/glastopf_log_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/dionaea/dionaea_log_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-10/dionaea/dionaea_file_2020-01-10_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-11/cowrie/cowrie_log_2020-01-11_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-11/cowrie/cowrie_file_2020-01-11_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-11/amun/amun_log_2020-01-11_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-11/amun/amun_file_2020-01-11_dir',
 '../data_hdd4t/中華電信/honeypot/2020-01-11/glastopf/glastopf_file_2020-01-11_dir',
 '../

In [23]:
%%time
pool_result = []
pool = Pool() #processes=21
for path in paths:
    r = pool.apply_async(parallel_untar,(path,))
    pool_result.append(r)
pool.close()
pool.join()
for r in pool_result:
    print('return:',r)

!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
!Error: b'' !
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4390>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4c90>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4290>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4490>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4690>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4850>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4310>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d163aa90>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d163ab90>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d16d4610>
return: <multiprocessing.pool.ApplyResult object at 0x7f10d163a750>
return: <multiprocessing.poo

'2020-01-11'

In [49]:


total, used, free = shutil.disk_usage(paths[0])

print("Total: %d GiB" % (total // (2**30)))
print("Used: %d GiB" % (used // (2**30)))
print("Free: %d GiB" % (free // (2**30)))

Total: 3667 GiB
Used: 2367 GiB
Free: 1113 GiB


In [34]:
if 1<2 and (2<3:
    print(1)

1


In [12]:
os.makedirs(pcap_path+new_dir_names[0], exist_ok=True)

In [5]:
!ls  ../

anaconda3  data  data_hdd4t  Download  Leo_code


In [25]:
cmd = ["ls",'../']
p = subprocess.Popen(cmd, stderr=subprocess.PIPE, stdout=subprocess.PIPE, shell=False)
out, err = p.communicate()
print(str(out))

b'anaconda3\ndata\ndata_hdd4t\nDownload\nLeo_code\n'


b''

spark test